In [1]:

import pandas as pd
import numpy as np
import folium
import geopandas
import geopy
from folium.plugins.marker_cluster import MarkerCluster
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
df = pd.read_csv('Final_Cleaned.csv', encoding='utf-8', sep=';')
df

,price,city,energy,area,land area,rooms,ejerudg,days active,latitude,longitude,...,distance to Odense,distance to city,closest city,residents,municipal,municipal tax rate,grundskyldspromille,property_type,sqm_price,Age
0,5195000.0,Valby,D,82.0,563.0,5.0,4894.0,87.0,55.652068,12.498816,...,136.137958,5.126456,Copenhagen,644431.0,Københavns,23.70,34.00,Villa,63353.658537,73.0
1,2595000.0,Sommersted,D,192.0,20011.0,7.0,2077.0,6.0,55.359367,9.255736,...,72.075634,72.075634,Odense,55340.0,Haderslev,26.30,24.36,Villa,13515.625000,93.0
2,3495000.0,Greve,D,116.0,794.0,4.0,3725.0,33.0,55.580640,12.286512,...,121.572639,20.688457,Copenhagen,50818.0,Greve,24.42,16.94,Villa,30129.310345,57.0
3,4500000.0,Borup,B,159.0,833.0,5.0,2862.0,229.0,55.492244,11.986446,...,101.536701,42.015572,Copenhagen,61718.0,Køge,25.26,21.04,Villa,28301.886792,15.0
4,1245000.0,Fredericia,D,157.0,1075.0,6.0,2101.0,105.0,55.540579,9.610832,...,51.888689,51.888689,Odense,51606.0,Fredericia,25.50,26.00,Villa,7929.936306,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,1695000.0,Hovedgård,C,78.0,828.0,2.0,1816.0,174.0,55.947943,9.964381,...,67.034720,28.205003,Aarhus,94443.0,Horsens,25.39,22.56,Villa,21730.769231,73.0
26523,695000.0,Faaborg,D,144.0,1028.0,4.0,1610.0,404.0,55.234210,10.476905,...,18.842811,18.842811,Odense,51849.0,Faaborg-Midtfyn,26.10,22.85,Villa,4826.388889,75.0
26524,695000.0,Bagenkop,E,112.0,700.0,4.0,1278.0,174.0,54.747589,10.674032,...,74.437042,74.437042,Odense,12316.0,Langeland,26.30,24.57,Villa,6205.357143,62.0
26525,1750000.0,Aabybro,C,198.0,696.0,7.0,2146.0,174.0,57.160593,9.735930,...,200.602518,16.770336,Aalborg,38187.0,Jammerbugt,25.70,32.85,Villa,8838.383838,56.0


In [2]:
df_geo = df.loc[:,['longitude','latitude','price']]
df_geo = df_geo.dropna()
denmark_center = (56.2639, 9.5018)

In [3]:
import folium
from folium.plugins import MarkerCluster

# Create a map
dk = folium.Map(
    location=denmark_center,
    tiles='cartodbpositron',
    zoom_start=7,
)

# Define the quantile breakpoints
quantiles = df['price'].quantile([0.33, 0.66])

# Define a color function based on price values
def color_function(price):
    if price < 1495000:  # Adjust these thresholds as needed
        return 'green'
    elif price < 2895000:
        return 'blue'
    else:
        return 'red'
df

,price,city,energy,area,land area,rooms,ejerudg,days active,latitude,longitude,...,distance to Odense,distance to city,closest city,residents,municipal,municipal tax rate,grundskyldspromille,property_type,sqm_price,Age
0,5195000.0,Valby,D,82.0,563.0,5.0,4894.0,87.0,55.652068,12.498816,...,136.137958,5.126456,Copenhagen,644431.0,Københavns,23.70,34.00,Villa,63353.658537,73.0
1,2595000.0,Sommersted,D,192.0,20011.0,7.0,2077.0,6.0,55.359367,9.255736,...,72.075634,72.075634,Odense,55340.0,Haderslev,26.30,24.36,Villa,13515.625000,93.0
2,3495000.0,Greve,D,116.0,794.0,4.0,3725.0,33.0,55.580640,12.286512,...,121.572639,20.688457,Copenhagen,50818.0,Greve,24.42,16.94,Villa,30129.310345,57.0
3,4500000.0,Borup,B,159.0,833.0,5.0,2862.0,229.0,55.492244,11.986446,...,101.536701,42.015572,Copenhagen,61718.0,Køge,25.26,21.04,Villa,28301.886792,15.0
4,1245000.0,Fredericia,D,157.0,1075.0,6.0,2101.0,105.0,55.540579,9.610832,...,51.888689,51.888689,Odense,51606.0,Fredericia,25.50,26.00,Villa,7929.936306,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,1695000.0,Hovedgård,C,78.0,828.0,2.0,1816.0,174.0,55.947943,9.964381,...,67.034720,28.205003,Aarhus,94443.0,Horsens,25.39,22.56,Villa,21730.769231,73.0
26523,695000.0,Faaborg,D,144.0,1028.0,4.0,1610.0,404.0,55.234210,10.476905,...,18.842811,18.842811,Odense,51849.0,Faaborg-Midtfyn,26.10,22.85,Villa,4826.388889,75.0
26524,695000.0,Bagenkop,E,112.0,700.0,4.0,1278.0,174.0,54.747589,10.674032,...,74.437042,74.437042,Odense,12316.0,Langeland,26.30,24.57,Villa,6205.357143,62.0
26525,1750000.0,Aabybro,C,198.0,696.0,7.0,2146.0,174.0,57.160593,9.735930,...,200.602518,16.770336,Aalborg,38187.0,Jammerbugt,25.70,32.85,Villa,8838.383838,56.0


In [4]:

folium.TileLayer('stamenterrain').add_to(dk)
# Iterate through the DataFrame rows and add markers with color based on prices
for index, row in df_geo.iterrows():
    price_color = color_function(row["price"])  # Get color based on price
    folium.CircleMarker(
        radius = 1,
        location=[row["latitude"], row["longitude"]],
        color=price_color,  # Set color based on price
        fill=True,
        fill_opacity=0.2,
    ).add_to(dk)

# Display the map
dk.save('colored_markers_map2.html')